## Setup

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext import data
from torchtext.vocab import GloVe

import random
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, classification_report

from IPython.display import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

## Parameters

### Architecture

In [8]:
# Embedding dimension/dimension for LSTM cell inputs
embed_dim = 300
# Number of hidden nodes
hidden_dim = 256
# Number of output nodes
output_dim = 1
# Number of LSTMs cells to be stacked
layers = 1
# Boolean value for bidirectioanl or not
bidirectional = False
# Boolean value for Xavier initialization
xavier_init = False

### Training

In [16]:
batch_size = 8
# Percentage of training data
split_ratio = 0.8
learning_rate = 0.001
epochs = 200

## Loading and preparing IMDB dataset

In [10]:
from imdb import IMDB_dataset
imdb = IMDB_dataset(split_ratio, seed)
imdb.load(verbose = True)
imdb.build_vocab(embed_dim)
train_loader, valid_loader, test_loader = imdb.create_data_loader(batch_size, device)
vocab_len = len(imdb.TEXT.vocab)

Training data size:    20000
Validation data size:  5000
Test data size:        25000


## Training model

In [11]:
# from lstm import LSTM
# model = LSTM(embed_dim, hidden_dim, layers=1, bidirectional=False, xavier_init=True)
# hidden_state = torch.zeros(1, batch_size, hidden_dim, requires_grad=True)
# cell_state = torch.zeros(1, batch_size, hidden_dim, requires_grad=True)
# inputs = torch.randn(20, batch_size, embed_dim)
# print(next(model.parameters()).is_cuda, inputs.is_cuda, hidden_state.is_cuda, cell_state.is_cuda)
# o, (h, c) = model(inputs, hidden_state, cell_state, device="cpu")
# print(h.shape)

In [12]:
# loss = torch.norm(h)
# loss.backward()
# optimizer = optim.Adam(model.parameters())
# optimizer.step()

In [ ]:
# from sentiment_analysis import *

model = LSTM_sentiment(vocab_len, embed_dim, hidden_dim, output_dim, 
                       imdb.pretrained_weights, layers, bidirectional, 
                       xavier_init)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_criterion = nn.BCEWithLogitsLoss()

# Train
model, stats = train_sentiment(model, train_loader, device, epochs, optimizer, 
                               loss_criterion, valid_loader)
print("=" * 50)

## Testing Model

In [ ]:
# Test
f1_test = evaluate_sentiment(model, test_loader, device, verbose=True)